First, we load the era5 data from copernicus api

In [1]:
import cdsapi
import xarray as xr

# Dane wejściowe
longitude = 16.3
latitude = 50.8

# Przygotowanie współrzędnych do użycia w nazwie pliku
lon_str = str(longitude).replace('.', '_')
lat_str = str(latitude).replace('.', '_')

# Dataset i zapytanie
dataset = "reanalysis-era5-land-timeseries"
request = {
    "variable": [
        "2m_dewpoint_temperature",
        "2m_temperature",
        "surface_pressure",
        "total_precipitation",
        "surface_solar_radiation_downwards",
        "snow_cover",
        "10m_u_component_of_wind",
        "10m_v_component_of_wind"
    ],
    "location": {"longitude": longitude, "latitude": latitude},
    "date": ["1950-01-02/2025-07-22"],
    "data_format": "netcdf"
}

# Ścieżka docelowa z podkreśleniami zamiast kropek
output_file = f"era5_all_{lon_str}_{lat_str}.zip"

# Pobranie danych
client = cdsapi.Client()
client.retrieve(dataset, request).download(output_file)

2025-08-19 09:50:43,873 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-08-19 09:50:44,086 WARNING [2025-06-04T00:00:00] This dataset provides user-selected location timeseries of [ERA5 Land data](https://doi.org/10.24381/cds.e2161bac) for a limited set of variables. Its content may be undergo changes over time (e.g. file format, data file structure, deprecation etc) and is **not recommended for use in a production environment**. 

For users interested in large regions, the original ERA5 Land catalogue entry remains the more efficient option. 
We will make every effort to notify users of changes, either through this banner and/or the [Forum](https://forum.ecmwf.int/).
2025-08-19 09:50:44,086 INFO Request ID is 964d8155-c9d0-42af-b663-e2af84d02c32
2025-08-19 09:50:44,172 INFO status has been updated to accepted
2025-08-19 09:50:57,830 INFO status has been updated to running
2025-08-19 09:51:05,470 INFO statu

'era5_all_16_3_50_8.zip'

Next, we want to process this file

In [2]:
import zipfile
import os
import xarray as xr

# Ścieżka do pliku ZIP i folder docelowy
zip_path = "era5_all_16_3_50_8.zip"
extract_dir = "era5_all_16_3_50_8"

# 1. Rozpakuj plik ZIP
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# 2. Znajdź pliki .nc
nc_files = [os.path.join(extract_dir, f) for f in os.listdir(extract_dir) if f.endswith('.nc')]

# 3. Wczytaj dane z plików .nc do słownika {nazwa_pliku: Dataset}
datasets = {}
for file in nc_files:
    name = os.path.splitext(os.path.basename(file))[0]  # np. "temperature"
    ds = xr.open_dataset(file)
    datasets[name] = ds

# 4. Przykład: wypisz dostępne zmienne
for name, ds in datasets.items():
    print(f"\n{name.upper()} variables:")
    print(ds.data_vars)


REANALYSIS-ERA5-LAND-TIMESERIES-SFC-2M-TEMPERATUREAGQ9WLQR variables:
Data variables:
    d2m      (valid_time) float32 3MB ...
    t2m      (valid_time) float32 3MB ...

REANALYSIS-ERA5-LAND-TIMESERIES-SFC-PRESSURE-PRECIPITATIONHN5CWLDD variables:
Data variables:
    sp       (valid_time) float32 3MB ...
    tp       (valid_time) float32 3MB ...

REANALYSIS-ERA5-LAND-TIMESERIES-SFC-RADIATION-HEATH_3QUSUL variables:
Data variables:
    ssrd     (valid_time) float32 3MB ...

REANALYSIS-ERA5-LAND-TIMESERIES-SFC-SNOW_C6NEHK7 variables:
Data variables:
    snowc    (valid_time) float32 3MB ...

REANALYSIS-ERA5-LAND-TIMESERIES-SFC-WIND9DB33WM0 variables:
Data variables:
    u10      (valid_time) float32 3MB ...
    v10      (valid_time) float32 3MB ...


And write to netcdf for later use

In [3]:
merged_ds = xr.merge(datasets.values())
merged_ds.to_netcdf("era5_all_16_3_50_8.nc")
merged_ds

<xarray.Dataset> Size: 26MB
Dimensions:     (valid_time: 662304)
Coordinates:
    latitude    float64 8B 50.8
    longitude   float64 8B 16.3
  * valid_time  (valid_time) datetime64[ns] 5MB 1950-01-02 ... 2025-07-22T23:...
Data variables:
    d2m         (valid_time) float32 3MB ...
    t2m         (valid_time) float32 3MB ...
    sp          (valid_time) float32 3MB ...
    tp          (valid_time) float32 3MB ...
    ssrd        (valid_time) float32 3MB ...
    snowc       (valid_time) float32 3MB ...
    u10         (valid_time) float32 3MB ...
    v10         (valid_time) float32 3MB ...
Attributes:
    Conventions:             CF-1.7
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_edition:            1
    GRIB_subCentre:          0
    history:                 2025-02-10T00:00 GRIB to CDM+CF via cfgrib-0.9.1...
    institution:             European Centre for Medium-Range Weather Forecasts